[Dan Larremore](http://twitter.com/danlarremore)

Feb 2, 2019

Notes on [Can you escape a maze without walls?](https://fivethirtyeight.com/features/can-you-escape-a-maze-without-walls/)

# Strategy

First of all, let's just solve the generic problem in order to solve this specific one. 

Second, let's reconsider the problem as an *escape* problem. You're at the smiley face, and you want to get out of the maze to the border. The behavior of S, U, and X squares is the same, but L squares send you right and R squares send you left. **We want to find an escape path from the goal to any boundary in the fewest number of hops**.

The key to both of the solutions below is that we'll split each grid square into what it really is: four nodes (top left bottom right), each of which has a different ability to connect to neighboring nodes. 

**Network Visualization Approach**
Why don't we just *build* this network and color the nodes by the type of node. If there is a connected path between a goal node and a boundary node, we'll just be able to see it. 

We'll use some code called [webweb](https://webwebpage.github.io) that makes network visualizations easy from Python, recently rewritten by my grad student, [Hunter Wapman](http://twitter.com/hneutr). 

The code plots the network as a grid, as described above, and it freeze the position of the nodes. **You can move the nodes around with your mouse if you like** and trace the escape path. I get a path length of 19. You can also uncheck the box that freezes node movement, and the system will "relax" allowing you to chart the number of hops from the Goal to the Boundary. Image here, followed by the code and notes to reproduce all of this.



**Network Traversal Approach (no viz)**
Number the positions of the grid like a matrix, with (1,1) being the upper left entry, and (n,n) being the bottom right entry. Now consider a network whose directed edges describe movement as dictated by the rules of the maze... but rather than considering each square in the gride as a single vertex, let each square in the grid be represented by four vertices, enumerated {1,2,3,4} corresponding to {up,left,down,right} directions. 

This means that we're going to take the $n^2$ positions in the grid and from them create $4n^2$ vertices in a network of one-hop moves. We'll also add another set of vertices representing the possible entry points to the game board from the outside, another $4n$ vertices. In total, our graph will have $N = 4n^2 + 4n$ vertices, or, if you've been taught to factor everything, $N = 4n(n+1)$ vertices. 

For convenience, we'll enumerate the vertices using 3 coordinates $(i,j,k)$ where $i$ and $j$ represent the position in the matrix (with row/column 0 and row/column $n$ being the boundaries) and $k \in \{1,2,3,4\}$ representing the four directions. (When this is coded up, we'll write 1-to-4 as 0-to-3 because python is zero-indexed).

Once we have the "forward" network, whose directed adjacency matrix is given by $A$---where $A_{uv}$ means that you can move from $u \to v$ in one step---our questions are generic network traversal questions. The transpose $B = A^{T}$ represents the movements _backward_ along the graph, and the entries of $B^t$ represent the number of paths there are from one node to another, backwards, in exactly $t$ steps. This means that we can consider the set of entries of $B^t$ that represent backward paths from **our goal** (a set of 4 nodes!) to **any boundary** (a set of 4n nodes!). Increment $t$ until one of those entries of $B^t$ is nonzero, and that's your answer to the puzzle. 

Note that the puzzle doesn't actually ask for the path, just the minimum number of steps, so we're good to use this method. 

Also note that _if_ the goal is unreachable, there will be no value of $t$ that allows us to move all the way through the graph. A cheap upper bound on $t$ should be $N-1$, since the longest traversal of the network would touch all nodes before getting to the goal, and I call this a cheap bound because $N$ includes the boundary nodes, which any maximum-length traversal would include only one of.


### Action of "L" Squares

Create the following edges:
$$
\begin{align}
    (i,j,1) &\to (i,j+1,2)\\
    (i,j,2) &\to (i-1,j,3)\\
    (i,j,3) &\to (i,j-1,4)\\
    (i,j,4) &\to (i+1,j,1)
\end{align}
$$

### Action of "R" Squares

Create the following edges:
$$
\begin{align}
    (i,j,1) &\to (i,j-1,4)\\
    (i,j,2) &\to (i+1,j,1)\\
    (i,j,3) &\to (i,j+1,2)\\
    (i,j,4) &\to (i-1,j,3)
\end{align}
$$

### Action of "U" Squares

Create the following edges:
$$
\begin{align}
    (i,j,1) &\to (i-1,j,3)\\
    (i,j,2) &\to (i,j-1,4)\\
    (i,j,3) &\to (i+1,j,1)\\
    (i,j,4) &\to (i,j+1,2)
\end{align}
$$

### Action of "X" Squares

Create no outgoing edges

### Action of "?" Squares

Let $V_{ij} = \{(i-1,j,3), (i,j-1,4), (i+1,j,1), (i,j+1,2) \}$

Then, create the following edges:
$$
\begin{align}
    (i,j,1) &\to V_{ij}\\
    (i,j,2) &\to V_{ij}\\
    (i,j,3) &\to V_{ij}\\
    (i,j,4) &\to V_{ij}
\end{align}
$$

# Code

In [63]:
import numpy as np
from webweb import Web

In [247]:
# We need something that can take in an array of symbols and generate a graph. 
# Let's store the graph as an edge list, i.e. (u,v) means an edge from u to v

# L - Suppose there is an L at position [i,j] in the grid? Return the list of edges to be added to the graph.
def get_edges_L(i,j):
    return [
        [[i,j,0],[i,j+1,1]], 
        [[i,j,1],[i-1,j,2]], 
        [[i,j,2],[i,j-1,3]], 
        [[i,j,3],[i+1,j,0]]]

# R - Suppose there is an R at position [i,j] in the grid? Return the list of edges to be added to the graph.
def get_edges_R(i,j):
    return [
        [[i,j,0],[i,j-1,3]], 
        [[i,j,1],[i+1,j,0]], 
        [[i,j,2],[i,j+1,1]], 
        [[i,j,3],[i-1,j,2]]]

# U - Suppose there is an U at position [i,j] in the grid? Return the list of edges to be added to the graph.
def get_edges_U(i,j):
    return [
        [[i,j,0],[i-1,j,2]], 
        [[i,j,1],[i,j-1,3]], 
        [[i,j,2],[i+1,j,0]], 
        [[i,j,3],[i,j+1,1]]]

# S - Suppose there is an S at position [i,j] in the grid? Return the list of edges to be added to the graph.
def get_edges_S(i,j):
    return [
        [[i,j,0],[i+1,j,0]], 
        [[i,j,1],[i,j+1,1]], 
        [[i,j,2],[i-1,j,2]], 
        [[i,j,3],[i,j-1,3]]]

# ? - Suppose there is an ? at position [i,j] in the grid? Return the list of edges to be added to the graph.
def get_edges_Q(i,j):
    vij = [[i-1,j,2],[i,j-1,3],[i+1,j,0], [i,j+1,1]]
    e = []
    here = [[i,j,0],[i,j,1],[i,j,2],[i,j,3]]
    for source in here:
        for destination in vij:
            e.append([source,destination])
    return e


# Also do boundaries:
def get_edge_boundary(i,j,n):
    # top
    if i==0:
        return [[[i,j,2],[i+1,j,0]]]
    # left
    if j==0: 
        return [[[i,j,3],[i,j+1,1]]]
    # bottom
    if i==n-1:
        return [[[i,j,0],[i-1,j,2]]]
    # right
    if j==n-1:
        return [[[i,j,1],[i,j-1,3]]]

# all together now
def get_edges(i,j,n,char):
    if char=='L':
        return get_edges_L(i,j)
    elif char=='R':
        return get_edges_R(i,j)
    elif char=='U':
        return get_edges_U(i,j)
    elif char=='S':
        return get_edges_S(i,j)
    elif char=='?':
        return get_edges_Q(i,j)
    elif char=='B':
        return get_edge_boundary(i,j,n)
    else:
        return []


In [248]:
def pad_grid(grid):
    grid = np.insert(grid,np.size(grid,0),'B',axis=0)
    grid = np.insert(grid,np.size(grid,1),'B',axis=1)
    grid = np.insert(grid,0,'B',axis=0)
    grid = np.insert(grid,0,'B',axis=1)
    grid[0,0] = ' '
    grid[0,-1] = ' '
    grid[-1,0] = ' '
    grid[-1,-1] = ' '
    for row in grid:
        for char in row:
            print('{} '.format(char),end='')
        print('')
    return grid

In [249]:
def grid2list(grid):
    n = np.size(grid,0)
    edgelist = []
    # do the board
    for i,row in enumerate(grid):
        for j,token in enumerate(row):
            edgelist += get_edges(i,j,n,token)
    return edgelist,rewrite_edgelist(edgelist,n,grid)

In [250]:
def triple2int(tr,n):
    return 4*(n*tr[1]+tr[0]) + tr[2]

In [251]:
def rewrite_edgelist(edgelist,n,grid):
    e = []
    for edge in edgelist:
        e += [[triple2int(edge[0],n),triple2int(edge[1],n)]]
    return e

In [310]:
def visualize_board(board):
    expanded_labels = {'B':' Boundary', 'G':' Goal','S':'Straight',
                       'L':'Left','R':'Right','X':'Dead End','?':'Any Direction','U':'U-turn'}
    board = pad_grid(board)
    raw,edgelist = grid2list(board)
    dim = 100*np.size(board,0)
    pad = 0.1
    visn = np.size(board,0)-1
    dxdy = {0:[0,-1], 1:[-1,0], 2:[0,1], 3:[1,0]}
    nodes = {}
    for idx,edge in enumerate(edgelist):
        i = raw[idx][0]
        label = board[i[0]][i[1]]
        nodenumber = edge[0]
        nodes[nodenumber] = {}
        nodes[nodenumber]['rule'] = expanded_labels[label]       
        nodes[nodenumber]['y'] = (i[0] + 0.1*dxdy[i[2]][1])/visn*(1-2*pad)*dim + pad*dim
        nodes[nodenumber]['x'] = (i[1] + 0.1*dxdy[i[2]][0])/visn*(1-2*pad)*dim + pad*dim

        j = raw[idx][1]
        label = board[j[0]][j[1]]
        nodenumber = edge[1]
        nodes[nodenumber] = {}
        nodes[nodenumber]['rule'] = expanded_labels[label] 
        nodes[nodenumber]['y'] = (j[0] + 0.1*dxdy[j[2]][1])/visn*(1-2*pad)*dim + pad*dim
        nodes[nodenumber]['x'] = (j[1] + 0.1*dxdy[j[2]][0])/visn*(1-2*pad)*dim + pad*dim
    web = Web(
        adjacency = edgelist,
        display ={
            'nodes' : nodes,
        }
    )
    web.display.colorBy = 'rule'
    web.display.colorPalette = 'Paired'
    web.display.freezeNodeMovement = True
    web.display.width = dim
    web.display.height = dim
    web.show()    

In [308]:
board = [
    ['U','L','X'],['U','G','?'],['U','L','X']]
visualize_board(board)

  B B B   
B U L X B 
B U G ? B 
B U L X B 
  B B B   


In [311]:
b538 = [
'LUU?ULXL',
'RLRLUGUU',
'SLRLULXR',
'UR?RSL?R',
'RUURRRSL',
'S?SLSSLR',
'RLR?RL?L',
'LRSRSLRL']
board538 = []
for row in b538:
    board538.append(list(row))
visualize_board(board538)

  B B B B B B B B   
B L U U ? U L X L B 
B R L R L U G U U B 
B S L R L U L X R B 
B U R ? R S L ? R B 
B R U U R R R S L B 
B S ? S L S S L R B 
B R L R ? R L ? L B 
B L R S R S L R L B 
  B B B B B B B B   


In [305]:
board538

[['L', 'U', 'U', '?', 'U', 'L', 'X', 'L'],
 ['R', 'L', 'R', 'L', 'U', 'G', 'U', 'U'],
 ['S', 'L', 'R', 'L', 'U', 'L', 'X', 'R'],
 ['U', 'R', '?', 'R', 'S', 'L', '?', 'R'],
 ['R', 'U', 'U', 'R', 'R', 'R', 'S', 'L'],
 ['S', '?', 'S', 'L', 'S', 'S', 'L', 'R'],
 ['R', 'L', 'R', '?', 'R', 'L', '?', 'L'],
 ['L', 'R', 'S', 'R', 'S', 'L', 'R', 'L']]